In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import csv
import pickle
from collections import Counter
from IPython.display import display
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, log_loss
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import RandomizedSearchCV
df = pd.read_csv('/content/drive/MyDrive/12th Grade/Machine Learning/Data/spaceshiptrain.csv')


In [ ]:
df.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [ ]:
df.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [ ]:
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [ ]:

from sklearn.impute import SimpleImputer

# Sample dataframe
# df = pd.DataFrame(...)
cat_columns = ['CryoSleep', 'VIP']
# For categorical columns
cat_imputer = SimpleImputer(strategy='most_frequent')
df[cat_columns] = cat_imputer.fit_transform(df[cat_columns])

# For numerical columns
num_columns = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
num_imputer = SimpleImputer(strategy='median')
df[num_columns] = num_imputer.fit_transform(df[num_columns])


In [ ]:




# Convert boolean columns to int
df['CryoSleep'] = df['CryoSleep'].astype(int)
df['VIP'] = df['VIP'].astype(int)



# Impute missing values in 'HomePlanet' with 'Mars'
df['HomePlanet'].fillna('Mars', inplace=True)

print(df)



     PassengerId HomePlanet  CryoSleep     Cabin    Destination   Age  VIP  \
0        0001_01     Europa          0     B/0/P    TRAPPIST-1e  39.0    0   
1        0002_01      Earth          0     F/0/S    TRAPPIST-1e  24.0    0   
2        0003_01     Europa          0     A/0/S    TRAPPIST-1e  58.0    1   
3        0003_02     Europa          0     A/0/S    TRAPPIST-1e  33.0    0   
4        0004_01      Earth          0     F/1/S    TRAPPIST-1e  16.0    0   
...          ...        ...        ...       ...            ...   ...  ...   
8688     9276_01     Europa          0    A/98/P    55 Cancri e  41.0    1   
8689     9278_01      Earth          1  G/1499/S  PSO J318.5-22  18.0    0   
8690     9279_01      Earth          0  G/1500/S    TRAPPIST-1e  26.0    0   
8691     9280_01     Europa          0   E/608/S    55 Cancri e  32.0    0   
8692     9280_02     Europa          0   E/608/S    TRAPPIST-1e  44.0    0   

      RoomService  FoodCourt  ShoppingMall     Spa  VRDeck     

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Instantiate the encoder
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data and convert to a dataframe
encoded_data = encoder.fit_transform(df[['HomePlanet']])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['HomePlanet']))

# Join the encoded data back with the original dataframe
final_df = df.join(encoded_df)

# Optionally, you can drop the original 'HomePlanet' column
final_df = final_df.drop('HomePlanet', axis=1)

print(final_df)


     PassengerId  CryoSleep     Cabin    Destination   Age  VIP  RoomService  \
0        0001_01          0     B/0/P    TRAPPIST-1e  39.0    0          0.0   
1        0002_01          0     F/0/S    TRAPPIST-1e  24.0    0        109.0   
2        0003_01          0     A/0/S    TRAPPIST-1e  58.0    1         43.0   
3        0003_02          0     A/0/S    TRAPPIST-1e  33.0    0          0.0   
4        0004_01          0     F/1/S    TRAPPIST-1e  16.0    0        303.0   
...          ...        ...       ...            ...   ...  ...          ...   
8688     9276_01          0    A/98/P    55 Cancri e  41.0    1          0.0   
8689     9278_01          1  G/1499/S  PSO J318.5-22  18.0    0          0.0   
8690     9279_01          0  G/1500/S    TRAPPIST-1e  26.0    0          0.0   
8691     9280_01          0   E/608/S    55 Cancri e  32.0    0          0.0   
8692     9280_02          0   E/608/S    TRAPPIST-1e  44.0    0        126.0   

      FoodCourt  ShoppingMall     Spa  

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
final_df

,PassengerId,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars
0,0001_01,0,B/0/P,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0.0,1.0,0.0
1,0002_01,0,F/0/S,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,1.0,0.0,0.0
2,0003_01,0,A/0/S,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0.0,1.0,0.0
3,0003_02,0,A/0/S,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0.0,1.0,0.0
4,0004_01,0,F/1/S,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0,A/98/P,55 Cancri e,41.0,1,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,0.0,1.0,0.0
8689,9278_01,1,G/1499/S,PSO J318.5-22,18.0,0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,1.0,0.0,0.0
8690,9279_01,0,G/1500/S,TRAPPIST-1e,26.0,0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,1.0,0.0,0.0
8691,9280_01,0,E/608/S,55 Cancri e,32.0,0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,0.0,1.0,0.0


In [ ]:
xcols = ['Age','CryoSleep','VIP','RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

In [ ]:
x = final_df[xcols]
x

,Age,CryoSleep,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,39.0,0,0,0.0,0.0,0.0,0.0,0.0
1,24.0,0,0,109.0,9.0,25.0,549.0,44.0
2,58.0,0,1,43.0,3576.0,0.0,6715.0,49.0
3,33.0,0,0,0.0,1283.0,371.0,3329.0,193.0
4,16.0,0,0,303.0,70.0,151.0,565.0,2.0
...,...,...,...,...,...,...,...,...
8688,41.0,0,1,0.0,6819.0,0.0,1643.0,74.0
8689,18.0,1,0,0.0,0.0,0.0,0.0,0.0
8690,26.0,0,0,0.0,0.0,1872.0,1.0,0.0
8691,32.0,0,0,0.0,1049.0,0.0,353.0,3235.0


In [ ]:
y = final_df['Transported']

In [ ]:
print(y)

0       False
1        True
2       False
3       False
4        True
        ...  
8688    False
8689    False
8690     True
8691    False
8692     True
Name: Transported, Length: 8693, dtype: bool


In [ ]:
print(x.columns)

Index(['Age', 'CryoSleep', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck'],
      dtype='object')


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
(x_train, x_test, y_train, y_test ) = train_test_split(x,y, stratify=y, test_size=0.2, random_state=0)

In [ ]:
(x_train, x_cv, y_train, y_cv) = train_test_split(x_train, y_train, stratify=y_train, test_size=0.20, random_state=0)

In [ ]:
x_train.shape

(5563, 8)

In [ ]:
y_train.shape

(5563,)

In [ ]:
scaling = MinMaxScaler()

In [ ]:
x_train

,Age,CryoSleep,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
4489,6.0,1,0,0.0,0.0,0.0,0.0,0.0
1218,28.0,0,1,0.0,0.0,0.0,0.0,0.0
1723,29.0,0,0,0.0,699.0,0.0,0.0,1955.0
3372,36.0,1,0,0.0,0.0,0.0,0.0,0.0
5349,27.0,1,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
3283,18.0,0,1,733.0,0.0,688.0,1.0,0.0
5520,25.0,0,0,0.0,0.0,892.0,0.0,0.0
1666,50.0,0,0,3677.0,782.0,0.0,6.0,1168.0
5444,2.0,1,0,0.0,0.0,0.0,0.0,0.0


In [ ]:
x_train = scaling.fit_transform(X=x_train)
x_cv = scaling.transform(X=x_cv)
x_test = scaling.transform(X=x_test)

In [ ]:
x

,Age,CryoSleep,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,39.0,0,0,0.0,0.0,0.0,0.0,0.0
1,24.0,0,0,109.0,9.0,25.0,549.0,44.0
2,58.0,0,1,43.0,3576.0,0.0,6715.0,49.0
3,33.0,0,0,0.0,1283.0,371.0,3329.0,193.0
4,16.0,0,0,303.0,70.0,151.0,565.0,2.0
...,...,...,...,...,...,...,...,...
8688,41.0,0,1,0.0,6819.0,0.0,1643.0,74.0
8689,18.0,1,0,0.0,0.0,0.0,0.0,0.0
8690,26.0,0,0,0.0,0.0,1872.0,1.0,0.0
8691,32.0,0,0,0.0,1049.0,0.0,353.0,3235.0


In [ ]:
#df['Destination'].unique()

In [ ]:
x_train

array([[7.59493671e-02, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.54430380e-01, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.67088608e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 9.61349331e-02],
       ...,
       [6.32911392e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 2.67761514e-04, 5.74350905e-02],
       [2.53164557e-02, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.65822785e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [ ]:
#def tuning(classifier, dist, X, y):
 # new_classifier = RandomizedSearchCV(estimator=classifier, random_state=0, param_distributions=dist, n_jobs=-1)
  #search = new_classifier.fit(X=X, y=y)
  #return search.best_params_

In [ ]:
#lr_model = LogisticRegression(random_state=0)

#lr_model = LogisticRegression()

In [ ]:
#lr_model.fit(x_train, y_train)

In [ ]:
#lr_model = LogisticRegression(random_state=0, max_iter=1000)

In [ ]:
#lr_model.fit(x_train, y_train)

In [ ]:
#sig_lr = CalibratedClassifierCV(base_estimator=lr_model)
#sig_lr.fit(X=x_train, y=y_train)

In [ ]:
#cv_pred = sig_lr.predict_proba(X=x_cv)

#cv_loss = log_loss(y_true=y_cv, y_pred=cv_pred)

In [ ]:
#cv_pred

In [ ]:
#prediction_cm_pred = sig_lr.predict(X=x_train)

In [ ]:
#print(classification_report(y_true=y_train, y_pred=prediction_cm_pred))

In [ ]:
#from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#initialize the classifier
#classifier = KNeighborsClassifier()

In [ ]:
#dist = dict(n_neighbors=[3, 5, 10, 15, 21, 25, 31, 41, 99])
#best_knn_model = tuning(classifier=classifier, dist=dist, X=x_train, y=y_train)

#print("Best n_neighbours: {}".format(best_knn_model))

In [ ]:
#classifier = KNeighborsClassifier(n_neighbors=best_knn_model['n_neighbors']).fit(X=x_train, y=y_train)

In [ ]:
#sig_classifier = CalibratedClassifierCV(base_estimator=classifier).fit(X=x_train, y=y_train)

In [ ]:

#prediction_cm_pred = sig_classifier.predict(X=x_train)

#print(classification_report(y_true=y_train, y_pred=prediction_cm_pred))


In [ ]:
#prediction_cm_pred = sig_classifier.predict(X=x_test)

#print(classification_report(y_true=y_test, y_pred=prediction_cm_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#classifier_rt = RandomForestClassifier(n_estimators=100, criterion='log_loss', max_depth=100, min_samples_split=5, random_state=30).fit(X=x_train, y=y_train)
#sig_classifier_rt = CalibratedClassifierCV(base_estimator=classifier_rt).fit(X=x_train, y=y_train)
#prediction_cm_pred = sig_classifier_rt.predict(X=x_train)

#print(classification_report(y_true=y_train, y_pred=prediction_cm_pred))

In [ ]:
classifier_rt = RandomForestClassifier(n_estimators=250, criterion='log_loss', max_depth=100, min_samples_split=5, random_state=30)
classifier_rt.fit(X=x_train, y=y_train)
sig_classifier_rt = CalibratedClassifierCV(base_estimator=classifier_rt).fit(X=x_train, y=y_train)
prediction_cm_pred = sig_classifier_rt.predict(X=x_train)

print(classification_report(y_true=y_train, y_pred=prediction_cm_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

       False       0.93      0.85      0.88      2762
        True       0.86      0.93      0.90      2801

    accuracy                           0.89      5563
   macro avg       0.89      0.89      0.89      5563
weighted avg       0.89      0.89      0.89      5563



In [ ]:
#classifier_rt = RandomForestClassifier(n_estimators=50, criterion='log_loss', max_depth=50, min_samples_split=10, random_state=30)
#classifier_rt.fit(X=x_train, y=y_train)
#sig_classifier_rt = CalibratedClassifierCV(base_estimator=classifier_rt).fit(X=x_train, y=y_train)
#prediction_cm_pred = sig_classifier_rt.predict(X=x_train)

#print(classification_report(y_true=y_train, y_pred=prediction_cm_pred))

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/12th Grade/Machine Learning/Data/spaceshuptest.csv')

In [ ]:
df_test

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [ ]:
df_test[num_columns] = num_imputer.transform(df_test[num_columns])
df_test[cat_columns] = cat_imputer.transform(df_test[cat_columns])



df_test[num_columns] = num_imputer.transform(df_test[num_columns])
df_test[cat_columns] = cat_imputer.transform(df_test[cat_columns])


In [ ]:
df_test.isna().sum()

PassengerId       0
HomePlanet       87
CryoSleep         0
Cabin           100
Destination      92
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name             94
dtype: int64

In [ ]:
df_test.shape

(4277, 13)

In [ ]:
df.shape

(8693, 14)

In [ ]:
df_test

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,27.0,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,27.0,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [ ]:
classifier_rt = RandomForestClassifier(n_estimators=250, criterion='log_loss', max_depth=100, min_samples_split=5, random_state=30)
classifier_rt.fit(X=x_train, y=y_train)
sig_classifier_rt = CalibratedClassifierCV(base_estimator=classifier_rt).fit(X=x_train, y=y_train)
prediction_cm_pred = sig_classifier_rt.predict(X=x_train)


#this is the test set testing
prediction = sig_classifier_rt.predict(X=x_test)

print(classification_report(y_true=y_test, y_pred=prediction))

/usr/local/lib/python3.10/dist-packages/sklearn/calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


              precision    recall  f1-score   support

       False       0.79      0.76      0.77       863
        True       0.77      0.80      0.79       876

    accuracy                           0.78      1739
   macro avg       0.78      0.78      0.78      1739
weighted avg       0.78      0.78      0.78      1739

